In [1]:
!pip install pandas openpyxl SQLAlchemy ollama -q

In [2]:
from google.colab import drive

# This command initiates the process to mount your Google Drive.
drive.mount('/content/drive')

Mounted at /content/drive


Get the Database ready

In [ ]:
import os

file_path = 'Drive Sample excel file path'

if os.path.exists(file_path):
    print(f"✅ Success! File found at: {file_path}")
else:
    print(f"❌ File not found. Please double-check the path you provided.")

✅ Success! File found at: /content/drive/MyDrive/IPRU test/Sample_Data.xlsx


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database File
db_file = 'sample_database.db'
engine = create_engine(f'sqlite:///{db_file}')

print(f"Reading all sheets from '{os.path.basename(file_path)}'...")

# Read all sheets Excel file into a dictionary of DataFrames
all_sheets_data = pd.read_excel(file_path, sheet_name=None)

print("\nStarting database creation...")
# Loop through each sheet and load it into a SQL table
for sheet_name, df in all_sheets_data.items():
    # Create a clean name for the SQL table (e.g., 'Sales Data' becomes 'sales_data')
    table_name = sheet_name.lower().replace(' ', '_')
    print(f"  -> Loading sheet '{sheet_name}' into table '{table_name}'...")
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"\n✅ Success! Database '{db_file}' is ready and all data has been loaded.")

Reading all sheets from 'Sample_Data.xlsx'...

Starting database creation...
  -> Loading sheet 'Dimension' into table 'dimension'...
  -> Loading sheet 'Application_Amount' into table 'application_amount'...
  -> Loading sheet 'Premium_Amount' into table 'premium_amount'...

✅ Success! Database 'IPRU_sample_database.db' is ready and all data has been loaded.


In [5]:
#test the DB

from sqlalchemy import text

test_query = "SELECT * FROM dimension LIMIT 5;"

print(f"Running test query: {test_query}")

try:
    with engine.connect() as connection:
        # We use pd.read_sql_query to run the query and get the result as a table
        result_df = pd.read_sql_query(text(test_query), connection)

    print("\nQuery successful! Here's a sample of your data from the database:")
    print(result_df)
except Exception as e:
    print(f"\n❌ An error occurred: {e}")
    print("Please check if the table name in your query is correct.")

Running test query: SELECT * FROM dimension LIMIT 5;

Query successful! Here's a sample of your data from the database:
   Contract_number Channel_attribute_4 Category_mis Sub_category_mis  \
0                2              Agency         Term     Jeevan Utsav   
1                3              Agency       Health      Jeevan Plus   
2                4             Offline         Ulip      Jeevan Labh   
3                5             Offline         Ulip     Jeevan Anand   
4                6             Offline       Health      Jeevan Plus   

     Segment Gender  
0  Segment 8   Male  
1  Segment 1      -  
2  Segment 6      -  
3  Segment 4   Male  
4  Segment 4      -  


Test the Local LLM using Ollama

In [6]:
# Part 1: Install Ollama
print("🚀 Starting Ollama installation...")
!curl -fsSL https://ollama.com/install.sh | sh

# Part 2: Start the Ollama server in the background
# 'nohup' and '&' ensure the server keeps running without blocking the notebook
print("\n🔥 Firing up the Ollama server in the background...")
!nohup ollama serve &

# Part 3: Give the server a moment to initialize
import time
print("Server is warming up. Please wait a few seconds...")
time.sleep(5)

# Part 4: Pull the codellama model
print("\n📥 Pulling the 'codellama' model. This may take a few minutes...")
!ollama pull codellama:7b

print("\n✅ Ollama server is running and the model is ready!")

🚀 Starting Ollama installation...
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

🔥 Firing up the Ollama server in the background...
nohup: appending output to 'nohup.out'
Server is warming up. Please wait a few seconds...

📥 Pulling the 'codellama' model. This may take a few minutes...


✅ Ollama server is running and the model is ready!


In [7]:
# This command displays the log file created by the Ollama server
!cat nohup.out

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIGFg5ljToWCb8iGN0Glz8bHM8BViAiM4ec4fnyBYplWQ

time=2025-09-09T05:57:12.926Z level=INFO source=routes.go:1331 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NEW_ESTIMATES:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://1

In [8]:
import ollama

try:
    # A simple test to see if the model responds
    print("🗣️ Sending a test message to the LLM...")
    response = ollama.chat(
        model='codellama:7b',
        messages=[
            {
                'role': 'user',
                'content': 'In one short sentence, why is the sky blue?',
            },
        ]
    )

    print("\n✅ Test successful! The LLM responded:")
    # We extract and print the content of the message from the response
    print(response['message']['content'])

except Exception as e:
    print(f"\n❌ An error occurred during the test: {e}")
    print("Please make sure the previous cell (Step 7) ran without errors.")

🗣️ Sending a test message to the LLM...

✅ Test successful! The LLM responded:

The sky appears blue because of a phenomenon called Rayleigh scattering, which occurs when sunlight enters Earth's atmosphere and scatters as it travels through the air molecules. This scattering effect is more pronounced for shorter wavelengths (such as blue light) than for longer wavelengths (such as red light), giving the sky its characteristic blue appearance.


Main Q&A engine

In [9]:
import pandas as pd
from sqlalchemy import create_engine, text
import ollama
import re

In [ ]:
# --- Make sure our database engine is still defined ---
db_file = 'sample_database.db'
engine = create_engine(f'sqlite:///{db_file}')

In [11]:
def get_schema(engine):
    """Gets the CREATE TABLE statements for the database to provide as context to the LLM."""
    with engine.connect() as connection:
        query = "SELECT sql FROM sqlite_master WHERE type='table';"
        schema = connection.execute(text(query)).fetchall()
        return "\n".join([s[0] for s in schema if s[0]])

In [20]:
def generate_sql(question: str, schema: str) -> str:
    """LLM Call 1: Generates the SQL query from a user question."""

    # We provide today's date for context-aware questions like "this month"
    today_date = "2025-09-09"

    prompt = f"""
You are an expert SQLite data analyst. First, think step-by-step about how to answer the user's question based on the schema. Deconstruct the question into the necessary SQL clauses (JOINs, WHERE, GROUP BY, etc.).
If the user's query requires columns from multiple tables, you MUST use a JOIN clause to link them via their common `Contract_number` which is the primary key.
After your step-by-step thinking, write a single, valid SQLite query to answer the question.
Only output the raw SQL query and nothing else. Do not wrap it in markdown or add explanations.

Here is an example of a good query that joins tables:
-- User Question:Average premium for health category in 2024
-- SQL Query: SELECT
  AVG(pa.'Issued_premium') AS 'Average Premium'
FROM dimension AS d
JOIN premium_amount AS pa
  ON d.'Contract_number' = pa.'Contract_number'
WHERE
  d.'Category_mis' = 'Health' AND pa.'Date' BETWEEN '2024-01-01' AND '2025-09-08';
THIS IS JUST AN EXAMPLE


Today's date is: {today_date}

Database Schema:
{schema}

User Question: {question}
"""

    response = ollama.chat(
        model='codellama:7b',
        messages=[{'role': 'user', 'content': prompt}],
        options={'temperature': 0.0} # Low temperature for factual, deterministic SQL
    )

    # Clean up the response to extract only the SQL
    sql_query = response['message']['content'].strip()
    # return re.sub(r"```sql\n|```", "", sql_query).strip()
    cleaned_query = re.sub(r"```sql\n|```|<s>|</s>", "", sql_query).strip()
    return cleaned_query

In [13]:
def run_query(sql_query: str, engine):
    """Executes the SQL query and returns the result as a pandas DataFrame."""
    try:
        with engine.connect() as connection:
            result_df = pd.read_sql_query(text(sql_query), connection)
        return result_df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

In [23]:
def interpret_result(question: str, data: pd.DataFrame) -> str:
    """LLM Call 2: Interprets the data and provides a natural language answer."""

    # Convert the DataFrame to a string format for the LLM
    data_string = data.to_string()

    prompt = f"""
You are a helpful data assistant. Based on the user's question and the following data, provide the user the answer in a nice natural language way.
The currency here is Indian Rupee '₹' make sure you get that right.

Original Question: {question}

Data:
{data_string}

Answer:
"""

    response = ollama.chat(
        model='codellama:7b',
        messages=[{'role': 'user', 'content': prompt}],
        options={'temperature': 0.5} # Higher temperature for more natural language
    )
    return response['message']['content'].strip()

In [24]:
# --- The Main Orchestrator Function ---

def ask_my_data(question: str, engine):
    """The main function that orchestrates the entire Text-to-SQL process."""
    print(f"🤔 Question: {question}")
    print("-" * 50)

    # Step 1: Get the database schema
    db_schema = get_schema(engine)

    # Step 2: Generate the SQL query (LLM Call 1)
    print("🤖 Generating SQL query...")
    sql_query = generate_sql(question, db_schema)

    try:
      result_df = run_query(sql_query, engine)
      print(f"🔍 Generated SQL:\n{sql_query}\n")

    except Exception as e:
      print("Initial query failed. Attempting to repair...")

      repair_prompt = f"""
      The following SQL query failed:
      {sql_query}

      The error message was:
      {e}

      Based on the schema below and the original question, please fix the query.

      Schema:
      {db_schema}

      Original Question: {question}

      Corrected SQL Query:
      """
      # Make another LLM call with the repair_prompt
      corrected_sql = ollama.chat(
        model='codellama:7b',
        messages=[{'role': 'user', 'content': repair_prompt}],
        options={'temperature': 0.0} # Low temperature for factual, deterministic SQL
    )

      # Try running the new query
      result_df = run_query(corrected_sql, engine)
      print(f"🔍 Generated corrected SQL:\n{corrected_sql}\n")


    # # Step 3: Execute the query
    # print("⚙️ Executing query against the database...")
    # result_df = run_query(sql_query, engine)

    if result_df is None or result_df.empty:
        print("No results found or an error occurred.")
        return

    print("📊 Raw Data Result:")
    print(result_df)
    print("\n")

    # Step 4: Interpret the result (LLM Call 2)
    print("💬 Interpreting result into natural language...")
    final_answer = interpret_result(question, result_df)

    print("\n💡 Final Answer:")
    print(final_answer)
    print("=" * 50 + "\n")

In [25]:
ask_my_data("What is the segment wise composition of 'Agency' channel?", engine)

🤔 Question: What is the segment wise composition of 'Agency' channel?
--------------------------------------------------
🤖 Generating SQL query...
🔍 Generated SQL:
SELECT d."Segment", COUNT(DISTINCT d."Contract_number") AS "Count"
FROM dimension d
JOIN application_amount a ON d."Contract_number" = a."Contract_number"
WHERE d."Channel_attribute_4" = 'Agency'
GROUP BY d."Segment";

📊 Raw Data Result:
     Segment  Count
0  Segment 1    392
1  Segment 2    409
2  Segment 3    351
3  Segment 4    406
4  Segment 5    367
5  Segment 6    403
6  Segment 7    372
7  Segment 8    365


💬 Interpreting result into natural language...

💡 Final Answer:
The segment wise composition of the 'Agency' channel is as follows:

* Segment 1 has a count of 392.
* Segment 2 has a count of 409.
* Segment 3 has a count of 351.
* Segment 4 has a count of 406.
* Segment 5 has a count of 367.
* Segment 6 has a count of 403.
* Segment 7 has a count of 372.
* Segment 8 has a count of 365.



In [26]:
ask_my_data("Total premium for Agency channel in from 2024 starting till today", engine)

🤔 Question: Total premium for Agency channel in from 2024 starting till today
--------------------------------------------------
🤖 Generating SQL query...
🔍 Generated SQL:
SELECT SUM(pa.'Issued_premium') AS 'Total Premium'
FROM dimension AS d
JOIN premium_amount AS pa ON d.'Contract_number' = pa.'Contract_number'
WHERE d.'Channel_attribute_4' = 'Agency' AND pa.'Date' BETWEEN '2024-01-01' AND '2025-09-09';

📊 Raw Data Result:
   Total Premium
0   1.395091e+08


💬 Interpreting result into natural language...

💡 Final Answer:
The total premium for the Agency channel from 2024 to today is ₹1,395,091.



In [19]:
ask_my_data("Segment composition of 'Agency' channel", engine)

🤔 Question: Segment composition of 'Agency' channel
--------------------------------------------------
🤖 Generating SQL query...
🔍 Generated SQL:
SELECT d."Segment", COUNT(*) AS "Count"
FROM dimension AS d
JOIN application_amount AS a ON d."Contract_number" = a."Contract_number"
WHERE d."Channel_attribute_4" = 'Agency'
GROUP BY d."Segment"
ORDER BY "Count" DESC;

📊 Raw Data Result:
     Segment  Count
0  Segment 2    409
1  Segment 4    406
2  Segment 6    403
3  Segment 1    392
4  Segment 7    372
5  Segment 5    367
6  Segment 8    365
7  Segment 3    351


💬 Interpreting result into natural language...

💡 Final Answer:
Hello! I'm happy to help you with your question about the segment composition of the 'Agency' channel.

Based on the data you provided, it seems that the most popular segments within this channel are Segments 2, 4, and 6, which account for a total of 1,398 users. These three segments have a combined user count of 1,398, while the next segment (Segment 1) has only 392